Use clustering, an unsupervised ML technique, to determine if a player's return ability is low, average, or high.

In [50]:
import pandas as pd
from catboost import CatBoostRegressor, Pool, cv
from sklearn.model_selection import train_test_split, KFold
import numpy as np
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

In [51]:
tennis_df = pd.read_excel("tennis.xlsx")

In [52]:
tennis_df.columns

Index(['X', 'tourney_id', 'tourney_name', 'surface', 'tourney_date',
       'match_date', 'match_num', 'round', 'best_of', 'gender', 'winner_id',
       'winner_seed', 'winner_rank', 'winner_name', 'winner_age',
       'winner_rating_score', 'winner_serve_ability', 'loser_id', 'loser_seed',
       'loser_rank', 'loser_name', 'loser_age', 'loser_rating_score',
       'loser_serve_ability', 'score', 'minutes', 'winner_aces',
       'winner_double_faults', 'winner_service_points',
       'winner_first_serves_in', 'winner_first_serves_won',
       'winner_second_serves_won', 'winner_break_points_saved',
       'winner_break_points_faced', 'winner_sets', 'winner_games',
       'loser_aces', 'loser_double_faults', 'loser_service_points',
       'loser_first_serves_in', 'loser_first_serves_won',
       'loser_second_serves_won', 'loser_break_points_saved',
       'loser_break_points_faced', 'loser_sets', 'loser_games'],
      dtype='object')

- winner aces
- winner break points saved
- winner break points faced
- loser break points faced
- loser break points saved
- winner first serves in - winner first serves won = winner first serves lost (loser successful return points)
- loser first serves in - loser first serves won = loser first serves lost (winer successful return points)


### create a table that has return information for clustering:
1. Name
2. X
3. match_num
4. W or L
5. break points earned (opposite of break points faced)
6. break points won
7. First serve returns won

In [ ]:
# create a function that calculates break points earned
# create a function that calculates break points won
# create a function that calculates first serve returns won

In [8]:
# Create a new df

return_df = pd.DataFrame()
loop_columns = ["winner_name", "loser_name"]

# Calculate break points earned:

def break_points_earned():
    return_df["break_points_earned"] = loser_break_points_faced

In [53]:
return_data = {"name":[],
               "X":[],
               "match_num":[],
               "WorL":[],
               "break_points_earned":[],
               "break_points_won":[],
               "first_serve_returns_won":[]}


In [54]:
def return_data_add(row):
    # Add winner names to the return_data dictionary
    return_data["name"].append(row["winner_name"])
    return_data["X"].append(row["X"])
    return_data["match_num"].append(row["match_num"])
    return_data["WorL"].append("W")
    return_data["break_points_earned"].append(row["loser_break_points_faced"])
    return_data["break_points_won"].append(row["loser_break_points_faced"]-row["loser_break_points_saved"])
    return_data["first_serve_returns_won"].append(row["loser_first_serves_in"]-row["loser_first_serves_won"])

    # Add loser names to the return_data dictionary
    return_data["name"].append(row["loser_name"])
    return_data["X"].append(row["X"])
    return_data["match_num"].append(row["match_num"])
    return_data["WorL"].append("L")
    return_data["break_points_earned"].append(row["winner_break_points_faced"])
    return_data["break_points_won"].append(row["winner_break_points_faced"]-row["winner_break_points_saved"])
    return_data["first_serve_returns_won"].append(row["winner_first_serves_in"]-row["winner_first_serves_won"])
    
    return return_data

In [55]:
return_data

{'name': [],
 'X': [],
 'match_num': [],
 'WorL': [],
 'break_points_earned': [],
 'break_points_won': [],
 'first_serve_returns_won': []}

In [56]:
return_data = {"name":[],
           "X":[],
           "match_num":[],
           "WorL":[],
           "break_points_earned":[],
           "break_points_won":[],
           "first_serve_returns_won":[]}

tennis_df.apply(return_data_add, axis=1)

0       {'name': ['Adrian Mannarino', 'Stefano Travagl...
1       {'name': ['Adrian Mannarino', 'Stefano Travagl...
2       {'name': ['Adrian Mannarino', 'Stefano Travagl...
3       {'name': ['Adrian Mannarino', 'Stefano Travagl...
4       {'name': ['Adrian Mannarino', 'Stefano Travagl...
                              ...                        
4567    {'name': ['Adrian Mannarino', 'Stefano Travagl...
4568    {'name': ['Adrian Mannarino', 'Stefano Travagl...
4569    {'name': ['Adrian Mannarino', 'Stefano Travagl...
4570    {'name': ['Adrian Mannarino', 'Stefano Travagl...
4571    {'name': ['Adrian Mannarino', 'Stefano Travagl...
Length: 4572, dtype: object

In [58]:
return_df = pd.DataFrame.from_dict(return_data)

In [60]:
return_df.head(50)

,name,X,match_num,WorL,break_points_earned,break_points_won,first_serve_returns_won
0,Adrian Mannarino,1,1123,W,12.0,8.0,22.0
1,Stefano Travaglia,1,1123,L,12.0,4.0,33.0
2,Alex De Minaur,2,1153,W,12.0,4.0,18.0
3,Bradley Klahn,2,1153,L,2.0,0.0,6.0
4,Alexander Bublik,3,1118,W,18.0,6.0,38.0
5,Rudolf Molleker,3,1118,L,8.0,3.0,14.0
6,Alexander Zverev,4,1116,W,17.0,7.0,38.0
7,John Millman,4,1116,L,13.0,7.0,31.0
8,Alexei Popyrin,5,1151,W,10.0,3.0,23.0
9,Ugo Humbert,5,1151,L,9.0,2.0,15.0
